## Pyspark Hands on

In [1]:
import pyspark
pyspark.__version__

'3.5.5'

In [2]:
from pyspark.sql import SparkSession

In [ ]:
# Initialize spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [ ]:
# Download fhvhv file
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [8]:
# unzip it
!gzip -d fhvhv_tripdata_2021-01.csv.gz

In [ ]:
# quick check for number of rows
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [ ]:
# read the file
df = spark.read.option("header", "true").csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
# .show() will trigger the execution of the dataframe.
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [13]:
# Create a smaller sample file for testing
!head -n 1000 fhvhv_tripdata_2021-01.csv > head.csv

In [14]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [10]:
import pandas as pd
pd.__version__

'2.2.3'

In [ ]:
# Creating a pandas df 
df_pandas = pd.read_csv("head.csv")

In [22]:
df_pandas.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,NaN
1,HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,NaN
2,HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,NaN
3,HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,NaN
4,HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,NaN


In [ ]:
# Spark can create a df from pandas df
spark_df = spark.createDataFrame(df_pandas)

In [26]:
spark_df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [ ]:
# Use this to check the schema of the dataframe
spark_df.schema

In [ ]:
from pyspark.sql import types

# defining the schema to be used for the spark df -> argument in .schema()
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)])


In [44]:
df = spark.read.option("header", "true").schema(schema).csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
# This is used for repartitioning the dataframe
df = df.repartition(24)

In [ ]:
# Write the dataframe to parquet format
df.write.parquet('fhvhv/2021/01/')

In [ ]:
# Read the parquet file created
df = spark.read.parquet('fhvhv/2021/01/')

In [51]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [ ]:
# Testing a filter (similar to SQL WHERE clause)
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-05 22:14:07|2021-01-05 22:32:28|         189|         107|
|2021-01-02 17:59:55|2021-01-02 18:10:39|          88|         137|
|2021-01-02 23:57:54|2021-01-03 00:15:48|         238|         224|
|2021-01-06 15:53:13|2021-01-06 16:07:07|         169|         208|
|2021-01-07 07:35:24|2021-01-07 07:55:49|          75|          88|
|2021-01-07 08:45:12|2021-01-07 08:51:17|         210|         210|
|2021-01-02 15:44:26|2021-01-02 16:10:50|         243|          69|
|2021-01-04 16:50:28|2021-01-04 16:57:43|         250|         213|
|2021-01-03 10:30:34|2021-01-03 10:44:53|          87|          79|
|2021-01-03 22:05:20|2021-01-03 22:27:55|          68|         181|
|2021-01-04 08:01:02|2021-01-04 08:33:27|          95|         236|
|2021-01-02 13:01:10|2021-01-02 13:08:11|       

- The same code in SQL
    ```sql
    SELECT 
        'pickup_datetime'
        ,'dropoff_datetime'
        ,'PULocationID'
        ,'DOLocationID'
    FROM
        df
    WHERE
        hvfhs_license_num == 'HV0003'
    ```

In [ ]:
from pyspark.sql import functions as F

df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date','dropoff_date','PULocationID','DOLocationID') \
    .show()


+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-03|  2021-01-03|         255|          34|
| 2021-01-05|  2021-01-05|         189|         107|
| 2021-01-02|  2021-01-02|          88|         137|
| 2021-01-02|  2021-01-03|         238|         224|
| 2021-01-06|  2021-01-06|         169|         208|
| 2021-01-07|  2021-01-07|          75|          88|
| 2021-01-07|  2021-01-07|         210|         210|
| 2021-01-02|  2021-01-02|         243|          69|
| 2021-01-04|  2021-01-04|         250|         213|
| 2021-01-03|  2021-01-03|          87|          79|
| 2021-01-03|  2021-01-03|          68|         181|
| 2021-01-04|  2021-01-04|          95|         236|
| 2021-01-02|  2021-01-02|         262|         236|
| 2021-01-04|  2021-01-04|         225|         233|
| 2021-01-06|  2021-01-06|         237|          83|
| 2021-01-05|  2021-01-05|         231|       

#### User Defined Functions ( udf )

In [ ]:
# This is how to use a custom function 

def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [ ]:
# eg.
crazy_stuff('B02884')


's/b44'

In [80]:
# define the python function as a udf
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())


In [78]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

[Stage 22:>                                                         (0 + 1) / 1]

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-03|  2021-01-03|         255|          34|
|  e/b42| 2021-01-05|  2021-01-05|         189|         107|
|  e/b33| 2021-01-02|  2021-01-02|          88|         137|
|  e/b38| 2021-01-02|  2021-01-03|         238|         224|
|  e/b3b| 2021-01-06|  2021-01-06|         169|         208|
|  e/b33| 2021-01-07|  2021-01-07|          75|          88|
|  e/acc| 2021-01-07|  2021-01-07|         210|         210|
|  e/acc| 2021-01-02|  2021-01-02|         243|          69|
|  e/b35| 2021-01-04|  2021-01-04|         250|         213|
|  s/b3d| 2021-01-03|  2021-01-03|          87|          79|
|  e/a39| 2021-01-03|  2021-01-03|          68|         181|
|  s/acd| 2021-01-04|  2021-01-04|          95|         236|
|  s/b13| 2021-01-02|  2021-01-02|         262|         236|
|  e/9ce| 2021-01-04|  2